# EG_births_model

E. Quinn 4/5/2021

Probability model for births

### Import standard python datascience packages

In [ ]:
import sys
import math
import re
import copy as cp
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

In [ ]:
from scipy.stats import nbinom
import numpy.random as npr

### Overview

This notebook builds a probability model for EG births using data from the 2020-2021 the NESDEC. 

It uses birth counts for the 18 years 2004-2020.

To summarize the results:

* A negative binomial model fits the data well.
* Medians, interquartile ranges, and 95%confidence intervals computed from the model are consistent with the data.
* The model extends to total births in a multiyear period. Results for multi-year models are consistent with the data for consecutive years. 

### Implications of the Model

* A model with independent, identical distributions of births in every year, while almost surely an oversimplification, never the less is a decent approximation; it captures the behavior well.
* The fact that the model fits the data well suggests that the expected number of births (population times birth rate) has remained fairly constant over the data period.  
* The observed variation is about what one would expect by pure chance, there are no outliers.
* We expect the number of births to fall between 96 and 114  50% of the time.
* We expect the number of births to fall between 85 and 127  90% of the time.
* We expect the number of births to fall between 81 and 131  95% of the time.

### Set path to data files

In [ ]:
data_path = '../'
!pwd

### Get EG Birth Counts

* Read births for 2005-2020 from 2020-2021 NESDEC data into an array
* Add 98 births for 2004 from Milone and MacBroom 2019 demographic study

In [ ]:
ta = np.genfromtxt(data_path+'NESDEC_2020_2021.csv', delimiter=",",skip_header=1)
births_array = np.empty((17,2))
births_array[0:17,0,] = np.arange(2004,2021)
births_array[0,1] = 98
births_array[1:17,1,] = ta[0:16,2,]
print(births_array)

### Get mean and variance of counts

In [ ]:
mu = np.mean(births_array[:,1,])
var = np.var(births_array[:,1,])

print('Mean births: ',mu,' births variance: ',var)

### Determine model and parameters

The variance is considerably larger than the mean, indicating *overdispersion*, so a Poisson model will not fit.

As and alternative, we consider a __[negative binomial distribution](https://en.wikipedia.org/wiki/Negative_binomial_distribution)__ with probability function:

$$P(Y=y) = {y + r -1 \choose y}(1-p)^{r}p^y,\quad y=0,1,2,3,...$$

We can estimate the parameters $r$ and $p$ using the *method of moments*.  We equate the sample mean and variance to the formulas for the expected value and variance of $y$:

$$ \frac{pr}{1-p} = 105.29 $$

$$ \frac{pr}{(1-p)^2} = 165.38$$

and solve the system for $r$ and $p$ to obtain:
* $\hat{p}$ = 0.3633
* $\hat{r}$ = 184.49

These are the parameters for a negative binomial random variable with mean $105.29$ and variance $165.38$.

### Compute method of moments estimates for r and p

In [ ]:
p = (165.38 - 105.29)/165.38
print('Method of moments estimate for p:',p)
r = 105.29*(1-p)/p
print('Method of moments estimate for r:',r)

### Define function for computing and printing percentiles

In [ ]:
def compute_percentiles(years,r,p,ymin,ymax):
    q1   = nbinom.ppf(0.25,years*r,1-p)               #first quartile
    q2   = nbinom.ppf(0.5,years*r,1-p)                #median
    q3   = nbinom.ppf(0.75,years*r,1-p)               #third quartile
    p05  = nbinom.ppf(0.05,years*r,1-p)               #lower bound - 90% CI           
    p95  = nbinom.ppf(0.95,years*r,1-p)               #upper bound - 90% CI                              
    p025 = nbinom.ppf(0.025,years*r,1-p)              #lower bound - 95% CI
    p975 = nbinom.ppf(0.975,years*r,1-p)              #upper bound - 95% CI
    plo  = round(nbinom.cdf(ymin,years*r,1-p),4)        #percentile of smallest observed value
    phi  = round(nbinom.cdf(ymax,years*r,1-p),4)       #percentile of largest observed value
    ptuple = (p025,p975,q1,q2,q3)                     #tuple of values for graphs

    print('Quartiles',q1,'median',q2,'third quartile',q3,'\n')
    print('Interquartile range',q3-q1,'\n')
    print('90 percent confidence interval:',p05,'-',p95,'  width:',p95-p05,'\n')
    print('95 percent confidence interval',p025,'-',p975,'  width:',p975-p025,'\n')
    print('Percentiles of largest and smallest observed values:  largest:',phi,' smallest:',plo)
    return(ptuple)

### Compute single-year quartiles and confidence interval bounds

With 18 observations, we expect 1.8 observations to fall outside a 90% CI.  In the sample, there were two.

We expect 0.9 observations to fall outside a 95% CI.  In this sample, one year landed on the lower bound, 81, so these results are in line with expectations.

In [ ]:
ptuple = compute_percentiles(1,r,p,81,126)

### Plot births data

In [ ]:
years = []
for y in births_array[:,0]:
    years.append(int(y))
births = births_array[:,1]
    
l95 = ptuple[0]*np.ones(len(years))
u95 = ptuple[1]*np.ones(len(years))
q1  = ptuple[2]*np.ones(len(years))
q2  = ptuple[3]*np.ones(len(years))
q3  = ptuple[4]*np.ones(len(years))

ax = plt.axes([0, 0, 1.78, 1.78])
ax.set_xticks(years)
ax.set_xticklabels(years, fontsize=12)
plt.plot(years, births)
plt.plot(years, u95, color='green', label='95% confidence interval (model)')
plt.plot(years, l95, color='green')
plt.plot(years, q1,color='red', label='Interquartile range (model)')
plt.plot(years, q2,color='black', label='Median (model)')
plt.plot(years, q3,color='red')
plt.legend(loc="upper left")
plt.ylim([0, 150])

ax.set_xlabel('Year', fontsize = 18)
ax.set_ylabel('EG Births', fontsize = 18)
ax.set_title('EG Births, quartiles, and 95% confidence limits', fontsize = 20)
plt.show()
plt.close()

### Derivation of the distribution of multi-year totals

If we assume that the births in successive years are independently distributed, we can derive the distribution of the total births in two or more years.

With the independence assumption, the moment generating function of the distribution of the sum will be the product of the moment generating functions for the years in the sum.

Since we have assumed the same distribution for the number of births every year, the common moment generating function is that of a negative binomial random variable with parameters $r$ and $p$:

$$ M_y(t) = \left(\frac{1-p}{1-pe^t}\right)^r$$

For the sum of two years, the moment generating function of the sum is the product:

$$ M_{2y}(t) = \left(\frac{1-p}{1-pe^t}\right)^r\left(\frac{1-p}{1-pe^t}\right)^r = \left(\frac{1-p}{1-pe^t}\right)^{2r}$$

which we can recognize as the moment generating function of a negative binomial random variable with parameters $2r$ and $p$.

We can extend this to sums of more than two years, the result being that the sum of $n$ independent negative binomial random variables with parameters $r$ and $p$ has a negative binomial distribution with parameters $n\cdot r$ and $p$. 

### Compute quartiles and confidence interval bounds for sums of two consecutive years

In [ ]:
ptuple = compute_percentiles(2,r,p,164,240)

### Plot births totals for consecutive years

In [ ]:
years = np.arange(2005,2021)
print(len(years))

data1 = births_array[0:len(years),1]
data2 = births_array[1:len(years)+1,1]
data=data1+data2

l95 = ptuple[0]*np.ones(len(years))
u95 = ptuple[1]*np.ones(len(years))
q1  = ptuple[2]*np.ones(len(years))
q2  = ptuple[3]*np.ones(len(years))
q3  = ptuple[4]*np.ones(len(years))

ax = plt.axes([0, 0, 1.78, 1.78])
ax.set_xticks(years)
ax.set_xticklabels(years, fontsize=12)
plt.plot(years, data)
plt.plot(years, u95, color='green', label='95% confidence interval (model)')
plt.plot(years, l95, color='green')
plt.plot(years, q1,color='red', label='Interquartile range (model)')
plt.plot(years, q2,color='black', label='Median (model)')
plt.plot(years, q3,color='red')
plt.legend(loc="upper left")
plt.ylim([0, 280])

ax.set_xlabel('Year', fontsize = 18)
ax.set_ylabel('EG Births in Two Consecutive Years', fontsize = 18)
ax.set_title('EG Births in Two Consecutive Years, Quartiles, and 95% Confidence Limits', fontsize = 20)
plt.show()
plt.close()

### Compute quartiles and confidence interval bounds for sums of three consecutive years

In [ ]:
ptuple = compute_percentiles(3,r,p,283,355)

### Plot births totals for three consecutive years

In [ ]:
years = np.arange(2006,2021)

data1 = births_array[0:len(years),1]
data2 = births_array[1:len(years)+1,1]
data3 = births_array[2:len(years)+2,1]
data=data1+data2+data3
    
l95 = ptuple[0]*np.ones(len(years))
u95 = ptuple[1]*np.ones(len(years))
q1  = ptuple[2]*np.ones(len(years))
q2  = ptuple[3]*np.ones(len(years))
q3  = ptuple[4]*np.ones(len(years))

ax = plt.axes([0, 0, 1.78, 1.78])
ax.set_xticks(years)
ax.set_xticklabels(years, fontsize=14)
plt.plot(years, data)
plt.plot(years, u95, color='green', label='95% confidence interval (model)')
plt.plot(years, l95, color='green')
plt.plot(years, q1,color='red', label='Interquartile range (model)')
plt.plot(years, q2,color='black', label='Median (model)')
plt.plot(years, q3,color='red')
plt.legend(loc="upper left")
plt.ylim([0, 410])

ax.set_ylabel('EG Births in Three Consecutive Years', fontsize = 18)
ax.set_xlabel('Year', fontsize = 18)
ax.set_title('EG Births in Three Consecutive Years, Quartiles, and 95% Confidence Limits', fontsize = 20)
plt.show()
plt.close()

### Compute four-year quartiles and confidence interval bounds

In [ ]:
ptuple = compute_percentiles(4,r,p,394,474)

### Plot births totals for four consecutive years

In [ ]:
years = np.arange(2007,2021)

data1 = births_array[0:len(years),1]
data2 = births_array[1:len(years)+1,1]
data3 = births_array[2:len(years)+2,1]
data4 = births_array[3:len(years)+3,1]
data=data1+data2+data3+data4
    
l95 = ptuple[0]*np.ones(len(years))
u95 = ptuple[1]*np.ones(len(years))
q1  = ptuple[2]*np.ones(len(years))
q2  = ptuple[3]*np.ones(len(years))
q3  = ptuple[4]*np.ones(len(years))

ax = plt.axes([0, 0, 1.78, 1.78])
ax.set_xticks(years)
ax.set_xticklabels(years)
plt.plot(years, data)
plt.plot(years, u95, color='green', label='95% confidence interval (model)')
plt.plot(years, l95, color='green')
plt.plot(years, q1,color='red', label='Interquartile range (model)')
plt.plot(years, q2,color='black', label='Median (model)')
plt.plot(years, q3,color='red')
plt.legend(loc="upper left")
plt.ylim([0, 540])

ax.set_ylabel('EG Births in Four Consecutive Years', fontsize = 18)
ax.set_title('EG Births in Four Consecutive Years, Quartiles, and 95% Confidence Limits', fontsize = 20)
plt.show()
plt.close()